In [86]:
import pandas as pd
import numpy as np
import regex as re
import pgeocode

In [87]:
df_crimes = pd.read_csv("../data/Neighbourhood_Crime_Rates.csv")

In [88]:
df_neighbourhoods = pd.read_csv("neighbourhoods_df.csv")

In [89]:
df_neighbourhoods.head()

,Postal Code,Latitude,Longitude,neighbourhood
0,M1B,43.806686,-79.194353,Malvern / Rouge
1,M1C,43.784535,-79.160497,Rouge Hill / Port Union / Highland Creek
2,M1E,43.763573,-79.188711,Guildwood / Morningside / West Hill
3,M1G,43.770992,-79.216917,Woburn
4,M1H,43.773136,-79.239476,Cedarbrae


In [90]:
df_neighbourhoods['hood_splits'] = df_neighbourhoods['neighbourhood'].apply(lambda x: x.split(' / '))

In [91]:
df_neighbourhoods.head()

,Postal Code,Latitude,Longitude,neighbourhood,hood_splits
0,M1B,43.806686,-79.194353,Malvern / Rouge,"[Malvern, Rouge]"
1,M1C,43.784535,-79.160497,Rouge Hill / Port Union / Highland Creek,"[Rouge Hill, Port Union, Highland Creek]"
2,M1E,43.763573,-79.188711,Guildwood / Morningside / West Hill,"[Guildwood, Morningside, West Hill]"
3,M1G,43.770992,-79.216917,Woburn,[Woburn]
4,M1H,43.773136,-79.239476,Cedarbrae,[Cedarbrae]


In [92]:
df_neighbourhoods['num_hood_splits'] = df_neighbourhoods['neighbourhood'].apply(lambda x: len(x.split(' / ')))
df_neighbourhoods.head()

,Postal Code,Latitude,Longitude,neighbourhood,hood_splits,num_hood_splits
0,M1B,43.806686,-79.194353,Malvern / Rouge,"[Malvern, Rouge]",2
1,M1C,43.784535,-79.160497,Rouge Hill / Port Union / Highland Creek,"[Rouge Hill, Port Union, Highland Creek]",3
2,M1E,43.763573,-79.188711,Guildwood / Morningside / West Hill,"[Guildwood, Morningside, West Hill]",3
3,M1G,43.770992,-79.216917,Woburn,[Woburn],1
4,M1H,43.773136,-79.239476,Cedarbrae,[Cedarbrae],1


In [93]:
def hood_split(hoods):
    n_splits = len(hoods.split(' / '))
    max_len = df_neighbourhoods.num_hood_splits.max()
    num_pads = max_len - n_splits
    lst = hoods.split(' / ')
    lst += [np.nan]*num_pads
    return lst

In [94]:
df_neighbourhoods["hood_splits_padded"] = df_neighbourhoods["neighbourhood"].apply(lambda x: hood_split(x))

In [95]:
df_neighbourhoods.head()

,Postal Code,Latitude,Longitude,neighbourhood,hood_splits,num_hood_splits,hood_splits_padded
0,M1B,43.806686,-79.194353,Malvern / Rouge,"[Malvern, Rouge]",2,"[Malvern, Rouge, nan, nan, nan, nan, nan, nan,..."
1,M1C,43.784535,-79.160497,Rouge Hill / Port Union / Highland Creek,"[Rouge Hill, Port Union, Highland Creek]",3,"[Rouge Hill, Port Union, Highland Creek, nan, ..."
2,M1E,43.763573,-79.188711,Guildwood / Morningside / West Hill,"[Guildwood, Morningside, West Hill]",3,"[Guildwood, Morningside, West Hill, nan, nan, ..."
3,M1G,43.770992,-79.216917,Woburn,[Woburn],1,"[Woburn, nan, nan, nan, nan, nan, nan, nan, nan]"
4,M1H,43.773136,-79.239476,Cedarbrae,[Cedarbrae],1,"[Cedarbrae, nan, nan, nan, nan, nan, nan, nan,..."


In [97]:
def sep_hood_columns(df):
    for i in range(len(df.hood_splits_padded[0])):
        df[f"hood_{i}"] = df['hood_splits_padded'].apply(lambda x: x[i])
    return df

In [99]:
sep_hood_columns(df_neighbourhoods).head()

,Postal Code,Latitude,Longitude,neighbourhood,hood_splits,num_hood_splits,hood_splits_padded,hood_0,hood_1,hood_2,hood_3,hood_4,hood_5,hood_6,hood_7,hood_8
0,M1B,43.806686,-79.194353,Malvern / Rouge,"[Malvern, Rouge]",2,"[Malvern, Rouge, nan, nan, nan, nan, nan, nan,...",Malvern,Rouge,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,43.784535,-79.160497,Rouge Hill / Port Union / Highland Creek,"[Rouge Hill, Port Union, Highland Creek]",3,"[Rouge Hill, Port Union, Highland Creek, nan, ...",Rouge Hill,Port Union,Highland Creek,NaN,NaN,NaN,NaN,NaN,NaN
2,M1E,43.763573,-79.188711,Guildwood / Morningside / West Hill,"[Guildwood, Morningside, West Hill]",3,"[Guildwood, Morningside, West Hill, nan, nan, ...",Guildwood,Morningside,West Hill,NaN,NaN,NaN,NaN,NaN,NaN
3,M1G,43.770992,-79.216917,Woburn,[Woburn],1,"[Woburn, nan, nan, nan, nan, nan, nan, nan, nan]",Woburn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M1H,43.773136,-79.239476,Cedarbrae,[Cedarbrae],1,"[Cedarbrae, nan, nan, nan, nan, nan, nan, nan,...",Cedarbrae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
df_neighbourhoods.drop(['neighbourhood','hood_splits','num_hood_splits','hood_splits_padded'], axis=1, inplace=True)

In [105]:
df_neighbourhoods.head()

,Postal Code,Latitude,Longitude,hood_0,hood_1,hood_2,hood_3,hood_4,hood_5,hood_6,hood_7,hood_8
0,M1B,43.806686,-79.194353,Malvern,Rouge,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,43.784535,-79.160497,Rouge Hill,Port Union,Highland Creek,NaN,NaN,NaN,NaN,NaN,NaN
2,M1E,43.763573,-79.188711,Guildwood,Morningside,West Hill,NaN,NaN,NaN,NaN,NaN,NaN
3,M1G,43.770992,-79.216917,Woburn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M1H,43.773136,-79.239476,Cedarbrae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def add_hoods(df):
    